In [1]:
import numpy as np
import tensorflow as tf
import cv2
import shutil
from pathlib import Path
from tensorflow.keras import layers, initializers, backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
# **Configuration**
INPUT_SHAPE = (224, 224, 3)  # RGB images
BATCH_SIZE = 8
EPOCHS = 20
LEARNING_RATE = 0.001
DATA_DIR = "D:/Major Project/Final Proper/indian+eye_yawn_dataset"
TRAIN_DIR = "D:/Major Project/Final Proper/train"
TEST_DIR = "D:/Major Project/Final Proper/test"
TEST_SIZE = 0.2  # 20% for testing

In [3]:
# import os
# import shutil
# import random

# # Define paths
# dataset_path = "D:/Major Project/Final Proper/indian+eye_yawn_dataset"
# train_path = "D:/Major Project/Final Proper/train"
# test_path = "D:/Major Project/Final Proper/test"

# # Define split ratio
# train_ratio = 0.8  # 80% training, 20% testing

# # Create train and test directories
# for category in os.listdir(dataset_path):
#     category_path = os.path.join(dataset_path, category)
    
#     if os.path.isdir(category_path):
#         images = os.listdir(category_path)
#         random.shuffle(images)  # Shuffle before splitting
        
#         train_size = int(len(images) * train_ratio)
        
#         train_images = images[:train_size]
#         test_images = images[train_size:]
        
#         # Create category folders in train and test directories
#         os.makedirs(os.path.join(train_path, category), exist_ok=True)
#         os.makedirs(os.path.join(test_path, category), exist_ok=True)
        
#         # Move files to train directory
#         for img in train_images:
#             shutil.copy2(os.path.join(category_path, img), os.path.join(train_path, category, img))
        
#         # Move files to test directory
#         for img in test_images:
#             shutil.copy2(os.path.join(category_path, img), os.path.join(test_path, category, img))

# print("Dataset split completed successfully!")


In [4]:
# # **Ensure Dataset is RGB**
# def convert_images_to_rgb():
#     for category in ["closed_eye", "open_eye"]:
#         img_dir = DATA_DIR / category
#         for img_path in img_dir.glob("*.png"):  # Adjust for jpg/jpeg if needed
#             img = cv2.imread(str(img_path))  # Reads in BGR (already 3 channels)
#             if img is None:
#                 continue
#             cv2.imwrite(str(img_path), img)  # Save back as RGB

# convert_images_to_rgb()

In [5]:
# **Custom Capsule Layers**
class Length(layers.Layer):
    def call(self, inputs, **kwargs):
        return K.sqrt(K.sum(K.square(inputs), -1) + K.epsilon())

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]

class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings

    def build(self, input_shape):
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]
        self.W = self.add_weight(
            shape=[1, self.input_num_capsule, self.num_capsule, self.dim_capsule, self.input_dim_capsule],
            initializer=initializers.glorot_uniform(),
            name='W'
        )
        self.built = True

    def call(self, inputs):
        inputs_expand = K.expand_dims(K.expand_dims(inputs, 2), 2)
        W_tiled = K.tile(self.W, [K.shape(inputs)[0], 1, 1, 1, 1])
        inputs_hat = tf.squeeze(tf.matmul(W_tiled, inputs_expand, transpose_b=True), axis=-1)
        b = tf.zeros(shape=[K.shape(inputs)[0], self.input_num_capsule, self.num_capsule])

        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=2)
            outputs = tf.reduce_sum(inputs_hat * K.expand_dims(c, -1), axis=1)
            if i < self.routings - 1:
                b += tf.reduce_sum(inputs_hat * K.expand_dims(c, -1), axis=-1)
        return outputs

# **Build MobileNet + Capsule Model**
class MobileNetCapsNet:
    def __init__(self, input_shape=(224, 224, 3)):
        self.input_shape = input_shape
        self.model = self._build_model()

    def _build_model(self):
        base_model = tf.keras.applications.MobileNetV2(
            input_shape=self.input_shape,
            include_top=False,
            weights='imagenet'
        )
        base_model.trainable = False  # Freeze MobileNet

        x = base_model.output
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Reshape((-1, 64))(x)
        x = CapsuleLayer(num_capsule=4, dim_capsule=8, routings=1)(x)  # Change num_capsule=4
        outputs = Length()(x)

        return tf.keras.Model(inputs=base_model.input, outputs=outputs)


# **Margin Loss Function**
def margin_loss(y_true, y_pred):
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=4)  # Change depth=4
    L = y_true * tf.square(tf.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * tf.square(tf.maximum(0., y_pred - 0.1))
    return tf.reduce_mean(tf.reduce_sum(L, axis=1))


In [6]:
# # **Prepare Training and Testing Data**
# def prepare_datasets():
#     train_dir = DATA_DIR.parent / "train"
#     test_dir = DATA_DIR.parent / "test"

#     # Create directories
#     for cls in ["closed_eye", "open_eye"]:
#         (train_dir/cls).mkdir(parents=True, exist_ok=True)
#         (test_dir/cls).mkdir(parents=True, exist_ok=True)

#         files = list((DATA_DIR/cls).glob("*"))
#         if not files:
#             raise ValueError(f"No files found in {DATA_DIR/cls}")

#         train_files, test_files = train_test_split(files, test_size=TEST_SIZE, random_state=42)

#         for f in train_files:
#             shutil.copy(f, train_dir/cls/f.name)
#         for f in test_files:
#             shutil.copy(f, test_dir/cls/f.name)

#     return train_dir, test_dir

In [8]:
# **Train the Model**
def train_model():
    train_dir, test_dir = TRAIN_DIR, TEST_DIR

    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale = 1.0 / 255,
        rotation_range = 15,  # Reduce rotation to keep eye structure  
        width_shift_range = 0.15,  # Reduce shift  
        height_shift_range = 0.15,  # Reduce shift  
        shear_range = 0.2,  
        zoom_range = 0.15,  # Reduce zoom  
        brightness_range = [0.9, 1.1],  # Keep lighting realistic  
        horizontal_flip = True,  # Keep, since it's natural  
        vertical_flip = False,  # REMOVE, unnatural for faces  
        fill_mode = "nearest",
        validation_split = 0.2  
        )


    train_gen = train_datagen.flow_from_directory(
        train_dir,
        target_size=INPUT_SHAPE[:2],
        batch_size=BATCH_SIZE,
        class_mode='categorical',  # Change from 'binary' to 'categorical'
        color_mode='rgb',
        subset='training'
    )

    val_gen = train_datagen.flow_from_directory(
        train_dir,
        target_size=INPUT_SHAPE[:2],
        batch_size=BATCH_SIZE,
        class_mode='categorical',  # Change from 'binary' to 'categorical'
        color_mode='rgb',
        subset='validation'
    )

    print("Class Indices:", train_gen.class_indices)

    model = MobileNetCapsNet().model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
        loss='categorical_crossentropy',  # Ensure correct loss function
        metrics=['accuracy']
    )

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=EPOCHS,
        verbose=1
    )

    # **Test Model**
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)
    test_gen = test_datagen.flow_from_directory(
        test_dir,
        target_size=INPUT_SHAPE[:2],
        batch_size=BATCH_SIZE,
        class_mode='categorical',  # Change from 'binary' to 'categorical'
        color_mode='rgb',
        shuffle=False
    )

    y_pred = np.argmax(model.predict(test_gen), axis=1)
    y_true = test_gen.classes 

    print("\nTest Metrics:")
    print(f"Accuracy: {np.mean(y_true == y_pred):.4f}")
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=['Eyeclose', 'Happy', 'Neutral', 'Yawn']))

    # **Save Model**
    model.save("drowsiness_lesser_india_plus_eye.keras")
    print("\nModel saved as 'drowsiness_lesser_india_plus_eye.keras' ✅")


In [9]:
train_model()

Found 2381 images belonging to 4 classes.
Found 593 images belonging to 4 classes.
Class Indices: {'Eyeclose': 0, 'Happy': 1, 'Neutral': 2, 'Yawn': 3}



D:\Major Project\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
298/298 ━━━━━━━━━━━━━━━━━━━━ 74s 238ms/step - accuracy: 0.6913 - loss: 0.8314 - val_accuracy: 0.4368 - val_loss: 1.1547
Epoch 2/20
298/298 ━━━━━━━━━━━━━━━━━━━━ 57s 190ms/step - accuracy: 0.7891 - loss: 0.6046 - val_accuracy: 0.5464 - val_loss: 1.0723
Epoch 3/20
298/298 ━━━━━━━━━━━━━━━━━━━━ 56s 189ms/step - accuracy: 0.8277 - loss: 0.5436 - val_accuracy: 0.5396 - val_loss: 1.0078
Epoch 4/20
298/298 ━━━━━━━━━━━━━━━━━━━━ 56s 189ms/step - accuracy: 0.8508 - loss: 0.5127 - val_accuracy: 0.5025 - val_loss: 1.0948
Epoch 5/20
298/298 ━━━━━━━━━━━━━━━━━━━━ 56s 189ms/step - accuracy: 0.8575 - loss: 0.5030 - val_accuracy: 0.5835 - val_loss: 0.9747
Epoch 6/20
298/298 ━━━━━━━━━━━━━━━━━━━━ 56s 189ms/step - accuracy: 0.8670 - loss: 0.4670 - val_accuracy: 0.5902 - val_loss: 0.9322
Epoch 7/20
298/298 ━━━━━━━━━━━━━━━━━━━━ 57s 191ms/step - accuracy: 0.8816 - loss: 0.4374 - val_accuracy: 0.5885 - val_loss: 1.0017
Epoch 8/20
298/298 ━━━━━━━━━━━━━━━━━━━━ 57s 190ms/step - accuracy: 0.8860 - loss: 0